In [1]:
import pandas as pd

#Apertura del dataset
df = pd.read_json('../datasets/shelflife.json')
df.head(5)

,barcode,brand,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days,source_rule,country,confidence,generated_on
0,7501234567897,Jumex,Néctar de durazno 1 L,juice-box,270.0,10.0,60.0,Heuristic: Juice boxed unopened ~6–9 m; opened...,Mexico,0.55,2025-10-08
1,7507890123451,Dolores,Sardinas en salsa 155 g,canned-fish,1095.0,NaN,NaN,FoodKeeper: Canned fish unopened ~2–5 y,Mexico,0.80,2025-10-08
2,7505678901233,La Villita,Queso gouda 300 g,cheese-hard,NaN,56.0,180.0,FoodKeeper: Hard cheeses ~3–6 w; freezer up to...,Mexico,0.75,2025-10-08
3,7500123456788,La Sierra,Frijol pinto 1 kg,beans-dry,730.0,NaN,NaN,FoodKeeper: Dry beans 1–2 y,Mexico,0.85,2025-10-08
4,7506789012342,San Juan,Huevo blanco 12 pzas,eggs,NaN,28.0,NaN,FoodKeeper: Eggs shell (fridge ~3–5 w),Mexico,0.90,2025-10-08


In [2]:
#Preparando el dataset para hacer la clasificacion 
df_clasificado = df.copy()
columnas_usar = ['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']
df_clasificado = df_clasificado[columnas_usar]

#Rellenar los valores null con ceros
columnas_numericas = ['shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']
df_clasificado[columnas_usar] = df_clasificado[columnas_usar].fillna(0)

df_clasificado.head(15)

,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days
0,Néctar de durazno 1 L,juice-box,270.0,10.0,60.0
1,Sardinas en salsa 155 g,canned-fish,1095.0,0.0,0.0
2,Queso gouda 300 g,cheese-hard,0.0,56.0,180.0
3,Frijol pinto 1 kg,beans-dry,730.0,0.0,0.0
4,Huevo blanco 12 pzas,eggs,0.0,28.0,0.0
5,Azúcar blanca 1 kg,sugar-white,1825.0,0.0,0.0
6,Frijoles refritos 430 g,canned-beans,1095.0,3.0,60.0
7,Sal fina 750 g,salt,3650.0,0.0,0.0
8,Tortillas de maíz 1 kg,tortillas-corn,3.0,7.0,60.0
9,Aguacate Hass,avocado,5.0,7.0,120.0


In [3]:
# Clasificacion en perecedero = 1 o no_perecedero = 0

def clasificar_perecedero(fila):
    categorias_no_perecederas = [
        'juice-box', 'canned-fish', 'beans-dry', 'sugar-white', 'salt',
        'soft-drinks', 'tea-bags', 'oil-vegetable', 'chips', 'rice-white-dry',
        'pasta-dry', 'canned-tomato', 'canned-meat', 'cookies', 'cereal-box',
        'coffee-ground', 'milk-uht', 'bottled-water', 'salsa-jarred',
        'toothpaste', 'shampoo', 'household-detergent'
    ]

    # Si pertenece a una categoría no perecedera 0
    if fila['category_off'] in categorias_no_perecederas:
        return 0
    # Si requiere de estar en un ambiente frio es 1
    elif (fila['shelf_life_fridge_days'] > 0 or fila['shelf_life_freezer_days'] > 0):
        return 1  # Perecedero (Se beneficia del frío)

    #El producto no va en frío pero su vida útil en despensa es muy corta
    elif fila['shelf_life_pantry_days'] < 20:
        return 1  # Perecedero por vida util muy corta
    else:
        return 0

# Aplicar la clasificación
df_clasificado['perecedero'] = df.apply(clasificar_perecedero, axis=1)


df_clasificado.tail(100)

,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days,perecedero
100,Leche semidescremada UHT 1L,milk-uht,90.0,0.0,0.0,0
101,Pasta dental 75 ml,toothpaste,0.0,0.0,0.0,0
102,Salsa verde 210 g,salsa-jarred,540.0,30.0,120.0,0
103,Queso gouda 300 g,cheese-hard,0.0,56.0,180.0,1
104,Yoghurt griego 150 g,yogurt,0.0,14.0,0.0,1
...,...,...,...,...,...,...
195,Azúcar estándar 2 kg,sugar-white,1825.0,0.0,0.0,0
196,Pasta fusilli 500 g,pasta-dry,730.0,0.0,0.0,0
197,Totopos 200 g,chips,150.0,0.0,0.0,0
198,Galletas de chocolate 110 g,cookies,365.0,0.0,0.0,0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

df_clasificado = pd.read_csv('../datasets/productos_clasificados.csv')

X = df_clasificado[['category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']]
y = df_clasificado['perecedero']

# Conversion de categorias a numeros para que el modelo pueda ejecutar el algoritmo
encoder = LabelEncoder()
X['category_off'] = encoder.fit_transform(X['category_off'])

# Separar los datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Entrenar modelo
modelo = RandomForestClassifier(n_estimators=500, random_state=42)
modelo.fit(X_train, y_train)

# Evaluar modelo
score = modelo.score(X_test, y_test)
print(f"Precisión: {score:.2f}")


/tmp/ipykernel_47387/2696058041.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['category_off'] = encoder.fit_transform(X['category_off'])


Precisión: 1.00


In [5]:
#Matriz de confusion
from sklearn.metrics import confusion_matrix, classification_report

y_pred = modelo.predict(X_test)

#Generacion de la matriz
matriz_confusion = confusion_matrix(y_test, y_pred)
print('Matriz de confusin :')
print(matriz_confusion)

Matriz de confusin :
[[23  0]
 [ 0 27]]


In [6]:
#Haciendo pruebas del modelo con datos nnunca vistos (completamnete nuevos)

# Los nuevos productos
productos_nuevos = [
    ["Plátano tabasco 1 kg", "fruit-fresh", 5, 0, 0],
    ["Tomate saladet 1 kg", "vegetable-fresh", 4, 0, 0],
    ["Limón agrio 1 kg", "fruit-fresh", 7, 0, 0],
    ["Aguacate hass 1 kg", "fruit-fresh", 3, 0, 0],
    ["Pan dulce 1 pza", "bakery-fresh", 3, 0, 0],
    ["Tortillas de maíz 1 kg", "tortilla", 5, 10, 30],
    ["Jamón de pavo 250 g", "meat-processed", 0, 10, 60],
    ["Yogur de fresa 1L", "yogurt", 0, 15, 0],
    ["Queso panela 400 g", "cheese-soft", 0, 14, 90],
    ["Refresco de cola 600 ml", "soda", 365, 0, 0],
    ["Aceite vegetal 1L", "oil-vegetable", 730, 0, 0],
    ["Café molido 250 g", "coffee-ground", 540, 0, 0],
    ["Sal yodada 1 kg", "salt", 1825, 0, 0],
    ["Harina de trigo 1 kg", "flour", 365, 0, 0],
    ["Frijol negro 1 kg", "beans-dry", 365, 0, 0],
    ["Pollo entero crudo 1 kg", "meat-raw", 0, 5, 180],
    ["Pescado mojarra 1 kg", "fish-fresh", 0, 3, 150],
    ["Atún enlatado 140 g", "canned-fish", 720, 0, 0],
    ["Cereal de maíz 500 g", "cereal", 365, 0, 0],
    ["Mantequilla 200 g", "butter", 0, 90, 180],
]

df_nuevos = pd.DataFrame(productos_nuevos, columns=[
    "product_name", "category_off", "shelf_life_pantry_days",
    "shelf_life_fridge_days", "shelf_life_freezer_days"
])

In [7]:
X_nuevos = df_nuevos[['category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']]

X_nuevos['category_off'] = encoder.fit_transform(X_nuevos['category_off'])

predicciones = modelo.predict(X_nuevos)
df_nuevos['perecedero_prediccion'] = predicciones

df_nuevos.head(10)

/tmp/ipykernel_47387/536309389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_nuevos['category_off'] = encoder.fit_transform(X_nuevos['category_off'])


,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days,perecedero_prediccion
0,Plátano tabasco 1 kg,fruit-fresh,5,0,0,0
1,Tomate saladet 1 kg,vegetable-fresh,4,0,0,0
2,Limón agrio 1 kg,fruit-fresh,7,0,0,0
3,Aguacate hass 1 kg,fruit-fresh,3,0,0,0
4,Pan dulce 1 pza,bakery-fresh,3,0,0,0
5,Tortillas de maíz 1 kg,tortilla,5,10,30,1
6,Jamón de pavo 250 g,meat-processed,0,10,60,1
7,Yogur de fresa 1L,yogurt,0,15,0,1
8,Queso panela 400 g,cheese-soft,0,14,90,1
9,Refresco de cola 600 ml,soda,365,0,0,0


In [8]:
# Agregar las etiquetas reales
etiquetas_reales = [1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0,0,1]
df_nuevos['perecedero_real'] = etiquetas_reales

# Comparar
df_nuevos['acierto'] = df_nuevos['perecedero_prediccion'] == df_nuevos['perecedero_real']

# Calcular precision con productos nuevos
precision_nuevos = df_nuevos['acierto'].mean()
print(f"Precisión con productos nuevos: {precision_nuevos:.2f}")

df_nuevos_guardar = df_nuevos[['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days', 'perecedero_real']]
ultimo_indice = pd.read_csv('../datasets/productos_clasificados.csv', usecols=['Unnamed: 0']).iloc[-1, 0]
df_nuevos_guardar.index = range(ultimo_indice + 1, ultimo_indice + 1 + len(df_nuevos))
df_nuevos_guardar.to_csv('../datasets/productos_clasificados.csv', mode='a', header=False, index=True)

df_nuevos.head(20)



Precisión con productos nuevos: 0.75


,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days,perecedero_prediccion,perecedero_real,acierto
0,Plátano tabasco 1 kg,fruit-fresh,5,0,0,0,1,False
1,Tomate saladet 1 kg,vegetable-fresh,4,0,0,0,1,False
2,Limón agrio 1 kg,fruit-fresh,7,0,0,0,1,False
3,Aguacate hass 1 kg,fruit-fresh,3,0,0,0,1,False
4,Pan dulce 1 pza,bakery-fresh,3,0,0,0,1,False
5,Tortillas de maíz 1 kg,tortilla,5,10,30,1,1,True
6,Jamón de pavo 250 g,meat-processed,0,10,60,1,1,True
7,Yogur de fresa 1L,yogurt,0,15,0,1,1,True
8,Queso panela 400 g,cheese-soft,0,14,90,1,1,True
9,Refresco de cola 600 ml,soda,365,0,0,0,0,True


In [9]:
def clasificador_nuevo_producto(nombre:str, categoria:str, 
                                vida_normal:int, vida_refrigeracion:int, vida_congelacion:int):
    
    ruta = '../datasets/productos_clasificados.csv'
    
    datos_producto = pd.DataFrame({
            'category_off': [categoria],  
            'shelf_life_pantry_days': [vida_normal],
            'shelf_life_fridge_days': [vida_refrigeracion],
            'shelf_life_freezer_days': [vida_congelacion]
        })

        
    datos_producto['category_off'] = encoder.fit_transform(datos_producto['category_off'])
    prediccion = modelo.predict(datos_producto)
    
    ultimo_indice = pd.read_csv(ruta, usecols=['Unnamed: 0']).iloc[-1, 0]
    
    datos_producto['perecedero'] = prediccion
    datos_producto['category_off'] = categoria
    datos_producto['product_name'] = nombre


    datos_producto = datos_producto[['product_name', 'category_off',
                                     'shelf_life_pantry_days', 'shelf_life_fridge_days',
                                     'shelf_life_freezer_days', 'perecedero']]
    
    datos_producto.index = [ultimo_indice + 1]


    # Guardar solo la nueva fila
    datos_producto.to_csv(ruta, mode='a', header=False, index=True)
        
    resultado = 'perecedero' if prediccion[0] == 1 else 'no perecedero'

    return f"El producto '{nombre}' es de tipo: {resultado}"

In [10]:
prueba_mango = clasificador_nuevo_producto(
    nombre="Mango Ataulfo 1kg",
    categoria="fruit-fresh",
    vida_normal=5,
    vida_refrigeracion=10,
    vida_congelacion=180
)
print(prueba_mango) 

El producto 'Mango Ataulfo 1kg' es de tipo: perecedero


In [11]:
prueba_manzana = clasificador_nuevo_producto(
    nombre="Manzana Gala en Bolsa 1kg",
    categoria="fruit-fresh",
    vida_normal=14,
    vida_refrigeracion=45,
    vida_congelacion=200
)
print(prueba_manzana)

El producto 'Manzana Gala en Bolsa 1kg' es de tipo: perecedero


In [12]:
prueba_pasta = clasificador_nuevo_producto(
    nombre="Pasta de Espagueti 500g",
    categoria="pasta-dry",
    vida_normal=730,
    vida_refrigeracion=0,
    vida_congelacion=0
)
print(prueba_pasta)

El producto 'Pasta de Espagueti 500g' es de tipo: no perecedero


In [13]:
prueba_galletas = clasificador_nuevo_producto(
    nombre="Galletas Saladas en Caja 300g",
    categoria="cookies",
    vida_normal=270,
    vida_refrigeracion=0,
    vida_congelacion=0
)
print(prueba_galletas)

El producto 'Galletas Saladas en Caja 300g' es de tipo: no perecedero


In [14]:
prueba_vino = clasificador_nuevo_producto(
    nombre="Vino Tinto Cabernet Sauvignon 750ml",
    categoria="beverage-alcoholic", # La nueva categoría que agregaste
    vida_normal=1095, # Vida útil sin abrir
    vida_refrigeracion=0,
    vida_congelacion=0
)
print(prueba_vino)

El producto 'Vino Tinto Cabernet Sauvignon 750ml' es de tipo: no perecedero
